In [ ]:
import os
import pandas as pd

# Set The Desired GPU Device for Server
runtime_gpu_id = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(runtime_gpu_id)

# CPU Thread Limitations for Server
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["MKL_NUM_THREADS"] = "4"
os.environ["VECLIB_MAXIMUM_THREADS"] = "4"
os.environ["NUMEXPR_NUM_THREADS"] = "4"

import tensorflow as tf

from dataset.dataset import CropDataset, CropDatasetCached
from model.vgg16 import Vgg16

limit_gb = 4
gpu_limit = 1024*limit_gb
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[runtime_gpu_id], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=gpu_limit)])

In [ ]:
stage = "Leaf1"
dfs_dir = "_dfs/"
cache_dir = "E:\cache_img"
level = 0
crop_size = (112, 112)
batch_size = 128
n_classes = 2

In [ ]:
stage_data = pd.read_csv(f"{dfs_dir}{stage}_data.csv")
dataset = CropDatasetCached(dataframe=stage_data, cache_dir=cache_dir, level=level, crop_size=crop_size, batch_size=batch_size, n_classes=n_classes)
dataset.process_and_cache_all_images()

In [ ]:
tf_dataset = dataset.get_dataset()

In [ ]:
dataset.plot_sample(1000)

In [ ]:
vgg16 = Vgg16(input_shape=(112, 112, 3), n_classes=2, first_trained_layer=11)
vgg16.compile(learning_rate=1e-5,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

# Configure early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20, verbose=1)

# Train the model, setting batch_size directly in model.fit()
vgg16.fit(tf_dataset,
            epochs=150,
            callbacks=[early_stopping])